In [26]:
import pandas as pd
import re

In [27]:
df = pd.concat(map(lambda x : pd.read_csv(f"ffkkmo_{x}_dump.csv"), ['2021', '2122', '2223', '2324', '2425'])).reset_index(drop=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33706 entries, 0 to 33705
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        33706 non-null  object 
 1   place       33706 non-null  object 
 2   online      33706 non-null  object 
 3   category    33706 non-null  object 
 4   segment     33706 non-null  object 
 5   rank        33706 non-null  int64  
 6   firstname   33706 non-null  object 
 7   middlename  1300 non-null   object 
 8   lastname    33706 non-null  object 
 9   club        33706 non-null  object 
 10  tss         33706 non-null  float64
 11  tes         33706 non-null  float64
 12  pcs         33706 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 3.3+ MB


In [29]:
df.head()

,date,place,online,category,segment,rank,firstname,middlename,lastname,club,tss,tes,pcs
0,27.09.2020,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3 юнoшеский разряд, мaльчики",Произвольная программа,1,Валентин,NaN,КРЫЛОВ,"г. Ивантеевка, МАУ ""СШ Лидер""",15.82,7.43,8.39
1,27.09.2020,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3 юнoшеский разряд, дeвочки",Произвольная программа,1,Елизавета,NaN,ИВАНОВА,"г. Сергиев Посад, МБУ СШ ""Центр""",20.04,9.28,10.76
2,27.09.2020,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3 юнoшеский разряд, дeвочки",Произвольная программа,2,Екатерина,NaN,МУРАЕВА,"г. Ивантеевка, МАУ ""СШ Лидер""",19.07,9.57,9.50
3,27.09.2020,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3 юнoшеский разряд, дeвочки",Произвольная программа,3,Милана,NaN,ВАСИЛЬЕВА,"г. Сергиев Посад, МБУ СШ ""Центр""",18.35,8.48,9.87
4,27.09.2020,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3 юнoшеский разряд, дeвочки",Произвольная программа,4,Элина,NaN,СОКОЛОВСКАЯ,"г.о. Балашиха, МАУ ""СШОР им. Ю.Е. Ляпкина""",17.99,9.11,9.38


In [30]:
# Приведем типы данных
df['date'] = df['date'].str.strip()
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

In [31]:
# Добавим год, месяц и сезон
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['season'] = df['date'].apply(lambda x: f"{x.year-2001}{x.year-2000}" if x.month < 8 else f"{x.year-2000}{x.year-1999}")

In [32]:
def replace_duplicate(df, keyword, name, field='club'):
    uniquenames = [ s for s in df[field].unique() if keyword.findall(str(s).lower()) ]
    df[field].replace(uniquenames, name, inplace=True)

In [33]:
def replace_duplicates(df, keywords, name, field='club'):
    for keyword in keywords:
        replace_duplicate(df, keyword, name, field)

In [34]:
def get_regex_cat(num, cat, gender):
    return re.compile((r"(" + num + ")(?s:.*?)(" + cat + ")(?s:.*?)(" + gender + ")").lower())

In [35]:
def get_regex_club(city, club):
    return re.compile((r"(" + club + ")(?s:.*?)(" + city + ")|(" + city + ")(?s:.*?)(" + club + ")").lower())

In [36]:
def get_regex(input_list):
    return re.compile((r"(" + ")(?s:.*?)(".join(input_list) + ")").lower())

In [37]:
def get_regex_exact(input_str):
    return re.compile((r"^" + input_str +"").lower())

In [38]:
def removeprefix(ss):
    if ss.startswith("г.о. "):
        return ss[len("г.о. "):]
    if ss.startswith("г. "):
        return ss[len("г. "):]

In [39]:
block_clubs = ['Белгородская область, МБУДО ДЮСШ по ЗВС',
 'Брянская область, СШ по ФКК ГАУ СК "Брянск"',
 'В\\К',
 'Владимирская область, МАУ СШ "Мотодром Арена"',
 'Владимирская область, МБУ СШОР г. Муром',
 'Воронежская область, ДЮСШ "Восточная" г. Лиски',
 'Воронежская область, ДЮСШ ФК "Хрустальный конёк"',
 'Воронежская область, МБУ СШОР №24',
 'ИП В. Азроян',
 'Калужская область, МБУ СШ "Космос" г. Калуга',
 'Костромская область, СШОР им. А.В. Голубева',
 'Курская область, АУ КО "Школа ЗВС "Арена"',
 'МБУ ДО БГО «СШОР»',
 'ООО «ХРУСТАЛЬНЫЙ ЛЁД»',
 'ООО «ХРУСТАЛЬНЫЙ ЛЕД»',
 'Рязанская область, ГАУ РО "СШ ДС "Олимпийский"',
 'Тверская область, ГБУ СШОР по ЛВС г. Тверь',
 'Тверская область, СШ ФФК г. Тверь',
 'Тульская область, ОКСШОР г. Тула',
 'Школа ФК Натальи Бестемьяновой',
 'Ярославская область, МУ СШОР №4',
 'Воронежская область, ДЮСШ ФК "Хрустальный конек"']

In [40]:
df.drop(df[df.club.isin(block_clubs)].index, inplace = True)

In [41]:
replace_duplicate(df, get_regex_club('', 'Ляпкина'), 'г.о. Балашиха, МАУ ДО «СШОР им. Ю.Е. Ляпкина»')
replace_duplicate(df, get_regex_club('', 'Ляпкна'), 'г.о. Балашиха, МАУ ДО «СШОР им. Ю.Е. Ляпкина»')
replace_duplicate(df, get_regex_club('', 'Голден'), 'г.о. Балашиха, СКФК «Голден Стар»')
replace_duplicate(df, get_regex_club('Балашиха', 'Олимп'), "г.о. Балашиха, АНО ДЮСШ «Олимп»")
replace_duplicate(df, get_regex_club('Мытищи', 'Авангард'), 'г.о. Мытищи, МБУ ДО «СШ «Авангард»')
replace_duplicate(df, get_regex_club('МБУ', 'Авангард'), 'г.о. Мытищи, МБУ ДО «СШ «Авангард»')
replace_duplicate(df, get_regex_club('Королев', 'Варяг'), 'г. Королев, СК «Варяг»')
replace_duplicate(df, get_regex_club('Королёв', 'Варяг'), 'г. Королев, СК «Варяг»')
replace_duplicate(df, get_regex_club('Королев', 'Сооруж'), 'г. Королев, МБУ «Спортивные Сооружения»')
replace_duplicate(df, get_regex_club('', 'Наши Надежды'), 'г.о. Красногорск, АНО ДО АФК «Наши Надежды»')
replace_duplicate(df, get_regex_club('', 'пахом'), 'г.о. Химки, АНО ДО «СШФК памяти Л.А. Пахомовой»')
replace_duplicate(df, get_regex_club('Наро-Фоминск', 'СШ'), 'г.о. Наро-Фоминский, МАУ ДО ЦСШ №1')
replace_duplicate(df, get_regex_club('Фоминск', 'МАУ'), 'г.о. Наро-Фоминский, МАУ ДО ЦСШ №1')
replace_duplicate(df, get_regex_club('Фомиск', 'МАУ'), 'г.о. Наро-Фоминский, МАУ ДО ЦСШ №1')
replace_duplicate(df, get_regex_club('Фомнск', 'МАУ'), 'г.о. Наро-Фоминский, МАУ ДО ЦСШ №1')
replace_duplicate(df, get_regex_club('Наро-Фоминск', 'Галактика'), 'г.о. Наро-Фоминск, ООО СКФК «Галактика»')
replace_duplicate(df, get_regex_club('Красногорск', 'Давыдов'), 'г.о. Красногорск, ИП Давыдов')
replace_duplicate(df, get_regex_club('Красногорск', 'КСШОР'), 'г.о. Красногорск, МБУ КСШОР "Зоркий"')
replace_duplicate(df, get_regex_club('', 'Зоркий'), 'г.о. Красногорск, МБУ КСШОР "Зоркий"')
replace_duplicate(df, get_regex_club('', 'Снежком'), 'г.о. Красногорск, ЦФМ «Снежком»')
replace_duplicate(df, get_regex_club('Красногорск', 'Снеж.ком'), 'г.о. Красногорск, ЦФМ «Снежком»')
replace_duplicate(df, get_regex_club('Красногорск', 'ЦФМ'), 'г.о. Красногорск, ЦФМ «Снежком»')
replace_duplicate(df, get_regex_club('Красногорск', 'DAVYDOV'), 'г.о. Красногорск, START TEAM DAVYDOV')
replace_duplicate(df, get_regex_club('Красногорск', 'Козачинский'), 'г.о. Красногорск, ИП Козачинский')
replace_duplicate(df, get_regex_club('Красногоск', 'Козачинский'), 'г.о. Красногорск, ИП Козачинский')
replace_duplicate(df, get_regex_club('Красногорск', 'Вербилло'), 'г.о. Красногорск, ИП Вербилло')
replace_duplicate(df, get_regex_club('Красногорск', 'Вербило'), 'г.о. Красногорск, ИП Вербилло')
replace_duplicate(df, get_regex_club('', 'Путь к успеху'), 'г.о. Красногорск, Союз МС Школа «Путь к успеху»')
replace_duplicate(df, get_regex_club('Красногорск', 'Эдвайс'), 'г.о. Красногорск, «ООО НК Эдвайс»')
replace_duplicate(df, get_regex_club('Люберцы', 'КСШОР'), 'г.о. Люберцы, МУДО КСШОР')
replace_duplicate(df, get_regex_club('Луберцы', ''), 'г.о. Люберцы, МУДО КСШОР')
replace_duplicate(df, get_regex_club('', 'Армия'), "г.о. Люберцы, СК «Армия фигурного катания»")
replace_duplicate(df, get_regex_club('', 'Армия фигурного катания'), "г.о. Люберцы, СК «Армия фигурного катания»")
replace_duplicate(df, get_regex_club('Лыткарино', ''), "г.о. Лыткарино, МАУ «ЛСК Лыткарино»")
replace_duplicate(df, get_regex_club('Ступино', 'Боброва'), "г. Ступино, МБУ ДО «СШОР им. В.Б. Боброва»")
replace_duplicate(df, get_regex_club('Щелково', ''), "г.о. Щелково, МАУ ГОЩ «ФОК «Ледовая арена» им. В.А. Третьяка")
replace_duplicate(df, get_regex_club('Щёлково', ''), "г.о. Щелково, МАУ ГОЩ «ФОК «Ледовая арена» им. В.А. Третьяка")
replace_duplicate(df, get_regex_club('Третьяка', ''), "г.о. Щелково, МАУ ГОЩ «ФОК «Ледовая арена» им. В.А. Третьяка")
replace_duplicate(df, get_regex_club('Щелкого', ''), "г.о. Щелково, МАУ ГОЩ «ФОК «Ледовая арена» им. В.А. Третьяка")
replace_duplicate(df, get_regex_club('Подольск', 'Витязь'), "г.о. Подольск, МУ ДО СШОР «Витязь»")
replace_duplicate(df, get_regex_club('Подольск', 'Арена'), "г.о. Подольск, ШФК «Арена»")
replace_duplicate(df, get_regex_club('Подольск', 'Слуцкой'), "г.о. Подольск, ШФК Слуцкой")
replace_duplicate(df, get_regex_club('Чехов', 'Витязь'), "г.о. Чехов, МАУ «СШ «Витязь» им. В.И. Васильева")
replace_duplicate(df, get_regex_club('Васильева', 'Витязь'), "г.о. Чехов, МАУ «СШ «Витязь» им. В.И. Васильева")
replace_duplicate(df, get_regex_club('Чехов', 'Спарта'), "г.о. Чехов, МАУ «СШ «Спарта»")
replace_duplicate(df, get_regex_club('', 'СЛУ-ГРУПП'), "г.о. Подольск, ООО «СЛУ-ГРУПП»")
replace_duplicate(df, get_regex_club('', 'Заречье'), "п. Заречье, ДЮСШ фигурного катания Ирины Слуцкой")
replace_duplicate(df, get_regex_club('ДЮСШ', 'Слуцкой'), "п. Заречье, ДЮСШ фигурного катания Ирины Слуцкой")
replace_duplicate(df, get_regex_club('', 'Лидер'), "г.о. Ивантеевка, МАУ ДО «СШ «Лидер»")
replace_duplicate(df, get_regex_club('', 'Салют'), "г.о. Долгопрудный, АУ «ФСК-Салют»")
replace_duplicate(df, get_regex_club('', 'альфа'), "г.о. Долгопрудный, СК «Альфа»")
replace_duplicate(df, get_regex_club('', 'Смирнова'), "г.о. Долгопрудный, АНО ДО СШФК «Смирнова Тим»")
replace_duplicate(df, get_regex_club('Клин', ''), "г. Клин, МБУ ДО СШОР «Клин спортивный»")
replace_duplicate(df, get_regex_club('Реутов', ''), "г.о. Реутов, Центр хоккея и фигурного катания «Айс Про»")
replace_duplicate(df, get_regex_club('', 'Айс Про'), "г.о. Реутов, Центр хоккея и фигурного катания «Айс Про»")
replace_duplicate(df, get_regex_club('', 'Ангелы'), "г.о. Одинцово, ООО «АФК «Ангелы Плющенко»")
replace_duplicate(df, get_regex_club('Одинцово', 'СШ'), "г.о. Одинцово, МБУ ДО СШ «Одинцово»")
replace_duplicate(df, get_regex_club('Одинцово', 'МБУС'), "г.о. Одинцово, МБУ ДО СШ «Одинцово»")
replace_duplicate(df, get_regex_club('Лобня', 'ЦФК'), "г.о. Лобня, МБУ «ЦФКиС»")
replace_duplicate(df, get_regex_club('Лобня', 'Дети'), "г.о. Лобня, СК ФК «Дети – наше будущее»")
replace_duplicate(df, get_regex_club('Дети', 'будущее'), "г.о. Лобня, СК ФК «Дети – наше будущее»")
replace_duplicate(df, get_regex_club('Лобня', 'Центр'), "г.о. Лобня, МБУ «ЦФКиС»")
replace_duplicate(df, get_regex_club('Лобня', 'УФК'), "г.о. Лобня, МБУ «ЦФКиС»")
replace_duplicate(df, get_regex_club('г.Лобня', ''), "г.о. Лобня, МБУ «ЦФКиС»")
replace_duplicate(df, get_regex_club('', 'Арктика'), 'г.о. Ленинский, АНО «ЦОСБЛ и ТК»')
replace_duplicate(df, get_regex_club('Видное', 'Центр'), "г.о. Ленинский, ООО «Центр спортивных технологий»")
replace_duplicate(df, get_regex_club('Видное', 'ЦСТ'), "г.о. Ленинский, ООО «Центр спортивных технологий»")
replace_duplicate(df, get_regex_club('Ленинский', 'ЦСТ'), "г.о. Ленинский, ООО «Центр спортивных технологий»")
replace_duplicate(df, get_regex_club('Ленинский', 'Центр'), "г.о. Ленинский, ООО «Центр спортивных технологий»")
replace_duplicate(df, get_regex_club('Видное', 'ЦОСБЛ'), "г.о. Ленинский, АНО «ЦОСБЛ и ТК»")
replace_duplicate(df, get_regex_club('Ленинский', 'ЦОСБЛ'), "г.о. Ленинский, АНО «ЦОСБЛ и ТК»")
replace_duplicate(df, get_regex_club('Можайск', ''), "г.о. Можайск, МУ «МДС «Багратион»")
replace_duplicate(df, get_regex_club('Электросталь', 'Стальной'), "г. Электросталь, СКФК «Стальной конек»")
replace_duplicate(df, get_regex_club('', 'Стальной конек'), "г. Электросталь, СКФК «Стальной конек»")
replace_duplicate(df, get_regex_club('', 'Кристалл'), "г. Электросталь, МБУДО «СШОР «Кристалл-Восток»")
replace_duplicate(df, get_regex_club('', 'ВОСТОК'), "г. Электросталь, МБУДО «СШОР «Кристалл-Восток»")
replace_duplicate(df, get_regex_club('Электросталь', 'ангел'), "г. Электросталь, ООО «КФК «Ледяной ангел»")
replace_duplicate(df, get_regex_club('Электросталь', 'Мартынова'), "г. Электросталь, ИП Мартынова")
replace_duplicate(df, get_regex_club('Селятино', 'Юность'), "п. Селятино, МАУ СШ №2 «Юность»")
replace_duplicate(df, get_regex_club('Раменское', ''), "г.о. Раменское, МБУ ДО «СШ «Раменское»")
replace_duplicate(df, get_regex_club('Люберцы', 'ХРУСТАЛЬНЫЙ'), "г.о. Люберцы, ООО «Хрустальный лёд»")
replace_duplicate(df, get_regex_club('Серги', 'Центр'), "г.о. Сергиев Посад, МБУ ДО СШ «Центр»")
replace_duplicate(df, get_regex_club('МБУ', '«Центр»'), "г.о. Сергиев Посад, МБУ ДО СШ «Центр»")
replace_duplicate(df, get_regex_club('Серги', 'ХРУСТАЛЬНЫЙ ЛЁД'), "г.о. Сергиев Посад, ООО «Хрустальный лед»")
replace_duplicate(df, get_regex_club('Серги', 'ХРУСТАЛЬНЫЙ ЛеД'), "г.о. Сергиев Посад, ООО «Хрустальный лед»")
replace_duplicate(df, get_regex_club('Серги', 'Стремление'), "г.о. Сергиев Посад, ООО «Стремление»")
replace_duplicate(df, get_regex_club('Домодедово', 'Спицын'), "г.о. Домодедово, ИП Спицин")
replace_duplicate(df, get_regex_club('Домодедово', 'Спицин'), "г.о. Домодедово, ИП Спицин")
replace_duplicate(df, get_regex_club('Домодедово', 'Авангард'), "г.о. Домодедово, МАУ «ГС «Авангард»")
replace_duplicate(df, get_regex_club('Домодедово', 'Бестемьяновой'), "г.о. Домодедово, ШФК Натальи Бестемьяновой")
replace_duplicate(df, get_regex_club('Коломна', 'Комета'), "г.о. Коломна, МБУ ДО СШОР «Комета»")
replace_duplicate(df, get_regex_club('Жуковский', 'Метеор'), "г.о. Жуковский, МБУ ДО «СШ – Центр спорта «Метеор»")
replace_duplicate(df, get_regex_club('', 'Демченко'), "г.о. Дмитровский, МУ ДО «СШОР Альберта Демченко»")
replace_duplicate(df, get_regex_club('Дмитров', 'Карпенко'), "г.о. Дмитровский, ИП Карпенко")
replace_duplicate(df, get_regex_club('Павлов', ''), "г.о. Павловский Посад, МУДО ДЮСШ")
replace_duplicate(df, get_regex_club('Серпухов', 'Спартак'), "г. Серпухов, МАУ Стадион «Спартак»")
replace_duplicate(df, get_regex_club('Серпухов', 'Золотой'), "г. Серпухов, СКФК «Золотой Конёк»")
replace_duplicate(df, get_regex_club('Серпухов', 'Благова'), "г. Серпухов, АНО АФК им. Василия Благова")
replace_duplicate(df, get_regex_club('Химки', 'ЗВС'), "г.о. Химки, МАУ ДО «СШ ЗВС»")
replace_duplicate(df, get_regex_club('', 'Бородул'), "г.о. Химки, СКФК Бородулиных")
replace_duplicate(df, get_regex_club('Ногинск', ''), "г. Ногинск, МБУ «СШОР»")
replace_duplicate(df, get_regex_club('Шаховская', ''), "г.о. Шаховская, МАУ «Шаховской ДОК»")
replace_duplicate(df, get_regex_club('', 'Алмазный'), "г.о. Подольск, СЦ «Алмазный лед МО»")
replace_duplicate(df, get_regex_club('Малаховка', ''), "г.о. Люберцы, СК «МГАФК - Малаховка»")
replace_duplicate(df, get_regex_club('Орехово', ''), "г. Орехово-Зуево, МУДО «СШ «Спартак Орехово»")
replace_duplicate(df, get_regex_club('ГБПОУ', 'УОР'), 'г.о. Краснознаменск, ГБПОУ МО «УОР № 1»')

/tmp/ipykernel_4805/490140345.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[field].replace(uniquenames, name, inplace=True)


In [42]:
df.club = df.club.apply(removeprefix)

In [43]:
cat_replace_dict = {
    "Юный фигурист, девочки": [
                        get_regex(["Юны", "", "д"]),
                        get_regex(["юн.фиг", "девочки"]),
                    ],
    "Юный фигурист, мальчики": [get_regex(["Юны", "", "м"])],
    "3-й юношеский разряд, мальчики": [
                       get_regex(["3", "юн", "м"]),
                       get_regex(["м", "3", "юн"])
                   ],
    "3-й юношеский разряд, девочки": [
                       get_regex(["3", "юн", "чк"]),
                       get_regex(["чк", "3", "юн"]),
                       get_regex(["Девушки", "3", "юношеский"]),
                   ],
    "2-й юношеский разряд, мальчики": [
                       get_regex(["2", "юн", "м"]),
                       get_regex(["м", "2", "юн"])
                   ],
    "2-й юношеский разряд, девочки": [
                       get_regex(["2", "юн", "шк"]),
                       get_regex(["2", "юн", "чк"]),
                       get_regex(["чк", "2", "юн"])
                   ],
    "1-й юношеский разряд, мальчики": [
                       get_regex(["1", "юн", "м"]),
                       get_regex(["м", "1", "юн"])
                   ],
    "1-й юношеский разряд, девочки": [
                       get_regex(["1", "юн", "чк"]),
                       get_regex(["чк", "1", "юн"]),
                       get_regex(["1", "юн", "шк"])
                   ],
    "3-й спортивный разряд, мальчики": [
                       get_regex(["ю", "3", "вн"]),
                       get_regex(["3", "вн", "ю"]),
                       get_regex(["м", "3", "вн"]),
                       get_regex(["3", "вн", "м"])
                   ],
    "3-й спортивный разряд, девочки": [
                       get_regex(["3", "вн", "чк"]),
                       get_regex(["3", "вн", "шк"]),
                       get_regex(["чк", "3", "вн"]),
                       get_regex(["шк", "3", "вн"]),
                       get_regex(["дeвочки", "3", "сп"]),
                       get_regex(["3", "сп", "девушки"])
                   ],
    "2-й спортивный разряд, мальчики": [
                       get_regex(["2", "вн", "м"]),
                       get_regex(["2", "вн", "ю"]),
                       get_regex(["м", "2", "вн"]),
                       get_regex(["ю", "2", "вн"]),
                       get_regex(["мaльчики", "3", "сп"])
                   ],
    "2-й спортивный разряд, девочки": [
                       get_regex(["2", "вн", "чк"]),
                       get_regex(["2", "вн", "шк"]),
                       get_regex(["чк", "2", "вн"]),
                       get_regex(["шк", "2", "вн"])
                   ],
    "1-й спортивный разряд, девочки": [
                       get_regex(["1", "вн", "чк"]),
                       get_regex(["1", "вн", "шк"]),
                       get_regex(["чк", "1", "вн"]),
                       get_regex(["шк", "1", "вн"])
                   ],
    "1-й спортивный разряд, мальчики": [
                       get_regex(["1", "вн", "м"]),
                       get_regex(["1", "вн", "ю"]),
                       get_regex(["м", "1", "вн"]),
                       get_regex(["ю", "1", "вн"])
                   ],
    "КМС, девушки": [get_regex_exact(x) for x in [
        'KMC, дeвушки', 'KMC, юниoрки', 'KMC, юниopки', 'КMC, юниoрки', 'KMC, девушки', 'КМС, дeвушки', 
        'KMС, юниopки','KМC, юниоpки', 'КМС дeвушки', 'КМС, юниорки', 'KMC, юниорки', 'КМС, юниoрки',
        'KMC, юниоpки', 'КМC юниоpки', 'КМС, юниоpки'
    ]],
    "КМС, юноши": [get_regex_exact(x) for x in ['KMC, юнoши', 'КМС, юниoры', 'КMС, юниoры', 'KMC, юниopы', 'KMC, юниoры',
       'КМС, юноши', 'КМС, юниоры','KMC, юниоpы', 'KMC ,юниоpы']],
    "МС, девушки": [get_regex_exact(x) for x in ['MC, дeвушки', 'MC, женщины', 'МС, девушки', 'MC, жeнщины', 'МC, жeнщины']],
    "МС, юноши": [get_regex_exact(x) for x in ['MC, мyжчины', 'MC, мужчины']],
    "Младшая группа, девочки": [get_regex([x]) for x in [
        'Дeвочки, млaдшая группа',
        'Млaдшaя гpуппa, дeвочки',
        'Млaдшaя гpуппа, дeвочки',
        'Дeвoчки, млaдшaя гpуппа',
        'Девочки, младшая группа',
        'Млaдшая группа, дeвочки',
        'Mлaдшaя группa, дeвoчки',
    ]],
    "Младшая группа, мальчики": [get_regex([x]) for x in [
        'Мaльчики, млaдшая группа',
        'Млaдшaя гpуппa, мaльчики',
        'Млaдшaя гpуппа, мaльчики',
        'Млaдшая группа, мaльчики',
        'Mлaдшaя группa, мaльчики', 
    ]],
    "Старшая группа, девочки": [get_regex([x]) for x in [
        'Дeвочки, cтаршая группа',
        'Cтapшая группa, дeвочки',
        'Cтapшaя гpуппа, дeвочки',
        'Cтapшaя гpуппa, дeвочки',
        'Cтapшая группа, дeвочки',
        'Cтaршaя группa, дeвoчки',
        'Cтaршaя группa, дeвoчкa',
    ]],
    "Старшая группа, мальчики": [get_regex([x]) for x in [
        'Мaльчики, cтаршая группа',
        'Cтapшая группa, мaльчики',
        'Cтapшaя гpуппа, мaльчики',
        'Cтapшaя гpуппa, мaльчики',
        'Cтapшая группа, мaльчики',
        'Cтaршaя группa, мaльчики',
        'Старшая группа, мальчики',
    ]],
}

In [44]:
for key in cat_replace_dict:
    replace_duplicates(df, cat_replace_dict[key], key, 'category')

/tmp/ipykernel_4805/490140345.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[field].replace(uniquenames, name, inplace=True)


In [45]:
df['category'].unique()

array(['3-й юношеский разряд, мальчики', '3-й юношеский разряд, девочки',
       '2-й юношеский разряд, мальчики', '2-й юношеский разряд, девочки',
       '1-й юношеский разряд, мальчики', '1-й юношеский разряд, девочки',
       '3-й спортивный разряд, мальчики',
       '3-й спортивный разряд, девочки', '1-й спортивный разряд, девочки',
       '1-й спортивный разряд, мальчики',
       '2-й спортивный разряд, девочки',
       '2-й спортивный разряд, мальчики', 'КМС, девушки', 'КМС, юноши',
       'Юный фигурист, девочки', 'Юный фигурист, мальчики', 'МС, девушки',
       'Младшая группа, девочки', 'Младшая группа, мальчики',
       'Старшая группа, девочки', 'Старшая группа, мальчики', 'Юнoши',
       'Дeвушки', 'юный фигурист', '«ЮНЫЙ ФИГУРИСТ»', 'МС, юноши',
       'Прoкат BK', 'Прокат ВК',
       'KMC, (члeны сборной Московской области)'], dtype=object)

In [46]:
df.drop(df[df.category.isin([
    'Юнoши',
    'Дeвушки',
    'юный фигурист',
    '«ЮНЫЙ ФИГУРИСТ»',
    'Прoкат BK',
    'Прокат ВК',
    'KMC, (члeны сборной Московской области)'
])].index, inplace = True)

In [47]:
df['category'].unique()

array(['3-й юношеский разряд, мальчики', '3-й юношеский разряд, девочки',
       '2-й юношеский разряд, мальчики', '2-й юношеский разряд, девочки',
       '1-й юношеский разряд, мальчики', '1-й юношеский разряд, девочки',
       '3-й спортивный разряд, мальчики',
       '3-й спортивный разряд, девочки', '1-й спортивный разряд, девочки',
       '1-й спортивный разряд, мальчики',
       '2-й спортивный разряд, девочки',
       '2-й спортивный разряд, мальчики', 'КМС, девушки', 'КМС, юноши',
       'Юный фигурист, девочки', 'Юный фигурист, мальчики', 'МС, девушки',
       'Младшая группа, девочки', 'Младшая группа, мальчики',
       'Старшая группа, девочки', 'Старшая группа, мальчики', 'МС, юноши'],
      dtype=object)

In [48]:
df.duplicated().sum()

2306

In [49]:
df = df.drop_duplicates()

In [50]:
df.reset_index(inplace=True, drop=True)
df.to_csv("ffkkmo_clean.csv")
df

,date,place,online,category,segment,rank,firstname,middlename,lastname,club,tss,tes,pcs,year,month,season
0,2020-09-27,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3-й юношеский разряд, мальчики",Произвольная программа,1,Валентин,NaN,КРЫЛОВ,"Ивантеевка, МАУ ДО «СШ «Лидер»",15.82,7.43,8.39,2020,9,2021
1,2020-09-27,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3-й юношеский разряд, девочки",Произвольная программа,1,Елизавета,NaN,ИВАНОВА,"Сергиев Посад, МБУ ДО СШ «Центр»",20.04,9.28,10.76,2020,9,2021
2,2020-09-27,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3-й юношеский разряд, девочки",Произвольная программа,2,Екатерина,NaN,МУРАЕВА,"Ивантеевка, МАУ ДО «СШ «Лидер»",19.07,9.57,9.50,2020,9,2021
3,2020-09-27,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3-й юношеский разряд, девочки",Произвольная программа,3,Милана,NaN,ВАСИЛЬЕВА,"Сергиев Посад, МБУ ДО СШ «Центр»",18.35,8.48,9.87,2020,9,2021
4,2020-09-27,"г. Лыткарино, ул. Советская д.9а, ЛСК «Лыткарино»",http://ffkkmo.ru/events/2021/lytkarino0927/,"3-й юношеский разряд, девочки",Произвольная программа,4,Элина,NaN,СОКОЛОВСКАЯ,"Балашиха, МАУ ДО «СШОР им. Ю.Е. Ляпкина»",17.99,9.11,9.38,2020,9,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31172,2025-06-14,"г. Долгопрудный, пр-кт Ракетостроителей, д. 4,...",http://ffkkmo.ru/events/2425/dp1406/,"Юный фигурист, девочки",Катание под музыку,3,Арина,Константиновна,ЕРЕМИНА,"Химки, МАУ ДО «СШ ЗВС»",11.73,5.43,6.30,2025,6,2425
31173,2025-06-14,"г. Долгопрудный, пр-кт Ракетостроителей, д. 4,...",http://ffkkmo.ru/events/2425/dp1406/,"Юный фигурист, девочки",Катание под музыку,4,Кристина,Александровна,ЛЕДУШКИНА,"Дмитровский, МУ ДО «СШОР Альберта Демченко»",10.60,4.80,5.80,2025,6,2425
31174,2025-06-14,"г. Долгопрудный, пр-кт Ракетостроителей, д. 4,...",http://ffkkmo.ru/events/2425/dp1406/,"Юный фигурист, девочки",Катание под музыку,5,Есения,Вячеславовна,СИДОРОВА,"Красногорск, МБУ КСШОР ""Зоркий""",9.43,4.13,5.30,2025,6,2425
31175,2025-06-14,"г. Долгопрудный, пр-кт Ракетостроителей, д. 4,...",http://ffkkmo.ru/events/2425/dp1406/,"Юный фигурист, девочки",Катание под музыку,6,Ульяна,Никитична,РОЙ,"Щелково, МАУ ГОЩ «ФОК «Ледовая арена» им. В.А....",7.39,2.19,5.20,2025,6,2425
